## Implementation: The Manifold

The manifold is the key concept for everything that follows, so it seems natural to start with that. Remember that we want to use test-driven development, so before we write code for any feature, we write a test. As a preliminary test list, I came up with this:

- Upon creation, the manifold should store its coordinates and metric
- For the Euclidean plane in cartesian coordinates, the Christoffel symbols should all be zero
- For the Euclidean plane in polar coordinates, the Christoffel symbols should not all be zero. The only ones that   do not vanish are

$$
\Gamma^r_{\;\,\varphi\varphi} =  -r\quad\quad \Gamma^\varphi_{\;\,r\varphi} = \Gamma^\varphi_{\;\,\varphi r} =  \frac{1}{r}
$$

#### Test: Upon creation, the manifold should store its coordinates and metric

Let's start with the first test. Create a file `tests/test_diffgeom.py` and write the following test code.

```python
import unittest
import sympy as sp
from diffgeom import Manifold


class TestManifold(unittest.TestCase):

    def test_init_manifold_stores_coords_and_metric(self):
        # Arrange
        coords = sp.symbols('x, y')
        metric = sp.diag(1, 1)
        # Act
        manifold = Manifold(metric, coords)
        # Assert
        self.assertEqual(manifold.coords, coords)
        self.assertEqual(manifold.metric, metric)
```


Running the test fails at this point, because we don't have any implementation yet! So let's now write just enough implementation to make the test pass. In `diffgeom/diffgeom.py` create a class:

```python
class Manifold(object):

    def __init__(self, metric, coords):
        self.metric = metric
        self.coords = coords


```

Running the test still fails:

```
ImportError: cannot import name 'Manifold' from 'diffgeom' (/Users/mbaer/PycharmProjects/diffgeom/diffgeom/__init__.py)
```

What's wrong? It does not find the class `Manifold` in the package `diffgeom`. That's because our class has the fully qualified name `diffgeom.diffgeom.Manifold`. But the statement `from diffgeom import Manifold` expects it to be `diffgeom.Manifold`. How can we do that? Well, just declare the name `Manifold` at the package level. Create a file `diffgeom/__init__.py` and write there

```python
from .diffgeom import Manifold
```

With this modification the test should run:

```
Ran 1 test in 0.002s

OK
```

Time to commit your changes!

```
git commit -am "Initializes Manifold object"
```

It is good practice to declare often used names at the package level. The main advantage is that this allows you to move the implementation to some other module. If users of the code just use the exported name, then moving the implementation does not break their code. However, if you don't export names, you force the users to use fully qualified names for your classes and functions and when you move the implementation, the code of all users breaks.



#### Test: Upon creation, the manifold should calculate the Christoffel symbols for its coordinate system

Let's move on to the next test! In `tests/test_diffgeom.py`, add another test to the `TestManifold` class:

```python
    def test_euclidean_plane_in_cartesian_coords_has_vanishing_christoffel(self):
        # Arrange
        coords = sp.symbols('x, y')
        metric = sp.diag(1, 1)
        # Act
        plane = Manifold(metric, coords)
        # Assert
        self.assertEqual(len(plane.gammas), 0)
```

Here we test if the data structure that will hold the Christoffel symbols has no entries. We only want to store non-vanishing components, because in practice most of the components would be zero anyways.

Again, running the test fails, because we haven't done the implementation yet! So let's do this now. Add the following code to the `diffgeom.diffgeom.Manifold.__init__` function:

```python
    self.gammas = {}
```

Running now all tests should succeed. But seriously? This is just a dummy implementation! Yes. But it is not unusual for test-driven development to write a dummy implementation until you are forced to have a really general one. "Fake it until you make it", so to speak. The approach is usually to write as little code as possible to satisfy all tests. And before writing a more general implementation, write more tests that force you to more generality.

All tests are passing, so commit and do some refactoring, if necessary.

#### Test: For the Euclidean plane in polar coordinates, the Christoffel symbols should not all be zero.

This test will force us to replace the dummy implementation with something more general. Add the following test to our test class:

```python
    def test_euclidean_plane_in_polar_coords_has_nonvanishing_christoffel(self):
        # Arrange
        r, phi = sp.symbols('r, phi')
        metric = sp.diag(1, r**2)
        # Act
        plane = Manifold(metric, coords=(r, phi)) # zeroth coord == r, first coord == phi
        # Assert
        self.assertEqual(len(plane.gammas), 3)
        self.assertEqual(plane.gammas[0, 1, 1], -r)
        self.assertEqual(plane.gammas[1, 0, 1], 1/r)
        self.assertEqual(plane.gammas[1, 1, 0], 1 / r)
```


Here I decided to store the Christoffel symbols in a dictionary, where the key values are integer index triples representing the coordinates. The zeroth coordinate is $r$ in our case and the first coordinate is $\varphi$. This is determined by the order of the coordinates given to the constructor. Since it is a bit cumbersome to remember the order of the coordinates, it would be nice if we had the option to access the Christoffel symbols optionally like so:

```python
plane.gammas[r, phi, phi]
```

We will implement that later. For now let's put this idea to our list of tests:

- Christoffel symbols should be accessible by coordinate names

So we don't forget it and implement it whenever it is convenient.

Runnung **all** tests will fail, since we don't have a general implementation for the Christoffel symbols, so let's turn to this in order to satisfy the tests.

The Christoffel symbols can be computed from the metric $g_{\mu\nu}$ and its inverse $g^{\mu\nu}$ from the following formula:

$$
\Gamma^{\mu}_{\;\alpha\beta} = 
\frac{1}{2}g^{\mu\nu}\left(
g_{\nu\beta, \alpha} + 
g_{\alpha\nu, \beta} -
g_{\alpha\beta, \nu}
\right)
$$

where we have an implicit summation over $\nu$ (Einstein sum convention!). Change the implemtation of the `diffgeom.Manifold` class to

```python
import sympy as sp


class Manifold(object):

    def __init__(self, metric, coords):
        self.metric = metric
        self.coords = coords
        self.gammas = self._calc_gammas()

    def _calc_gammas(self):
        x = self.coords
        g = self.metric
        gammas = {}
        n = len(x)
        g_inv = g.inv()
        for mu in range(n):
            for alpha in range(n):
                for beta in range(n):
                    gamma = 0
                    for nu in range(n):
                        gamma += g_inv[mu, nu] * (sp.diff(g[alpha, nu], x[beta]) +
                                                  sp.diff(g[nu, beta], x[alpha]) -
                                                  sp.diff(g[alpha, beta], x[nu])
                                                  )
                    if gamma != 0:
                        gammas[(mu, alpha, beta)] = gamma / 2
        return gammas

```

Here we have used the partial differentation function `diff` from `sympy`. Note that we only nonzero values.

Running **all** tests should succeed now. So we are ready to commit again.